# Test Load Daily

Start by loading one "new" tile.

## Imports

In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.table import Table
from sqlalchemy.exc import IntegrityError
from desiutil.log import get_logger, DEBUG
from desispec.io.meta import findfile
import specprodDB.load as db
import specprodDB.tile as t

## Initial Values

In [4]:
specprod = os.environ['SPECPROD']
# tile_id, tile_survey, tile_program = 3867, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5053, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5052, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5074, 'main', 'dark'
# tile_id, tile_survey, tile_program = 1685, 'main', 'dark'
# tile_id, tile_survey, tile_program = 40069, 'main', 'backup'
# tile_id, tile_survey, tile_program = 80950, 'sv1', 'backup'
tile_id = 80615
overwrite = True
tiles_patch_date = '20240906'

## Initialize Database

In [3]:
db.log = get_logger(DEBUG)
postgresql = db.setup_db(schema=specprod, hostname='db-loadbalancer.bweaver.development.svc.spin.nersc.org', username='desi_admin', overwrite=overwrite)
if overwrite:
    db.load_versions('computed', 'daily/v0', 'daily', specprod, 'main')

INFO:load.py:1794:setup_db: Begin creating tables.
INFO:load.py:1797:setup_db: Finished creating tables.
INFO:load.py:1540:load_versions: Loading version metadata.
INFO:load.py:1551:load_versions: Completed loading version metadata.


## Read tiles file

In [5]:
# tiles_file = findfile('tiles', readonly=True).replace('.fits', '.csv')
tiles_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], f'tiles-daily-patched-with-jura-{tiles_patch_date}.csv')
tiles_table = Table.read(tiles_file, format='ascii.csv')
# tiles_table

In [6]:
row_index = np.where((tiles_table['LASTNIGHT'] >= 20201214) & (tiles_table['EFFTIME_SPEC'] > 0) & (tiles_table['TILEID'] == tile_id))[0]

In [7]:
%%time
candidate_tiles = db.Tile.convert(tiles_table, row_index=row_index)

CPU times: user 595 µs, sys: 0 ns, total: 595 µs
Wall time: 600 µs


## Read exposures file

The daily exposures file may contain exposures with `EFFTIME_SPEC == 0`. We do not want to load these. There are also cases where a *tile* has non-zero `EFFTIME_SPEC` but has no *exposures* with non-zero `EFFTIME_SPEC`. At least for now, don't try to load those either.

In [8]:
# exposures_file = findfile('exposures', readonly=True)
exposures_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], f'exposures-daily-patched-with-jura-{tiles_patch_date}.fits')
exposures_table = Table.read(exposures_file, format='fits', hdu='EXPOSURES')
frames_table = Table.read(exposures_file, format='fits', hdu='FRAMES')
# exposures_table[exposures_table['TILEID'] == new_tile.tileid]

In [9]:
%%time
load_tiles = list()
bad_tiles = list()
load_exposures = list()
for new_tile in candidate_tiles:
    row_index = np.where((exposures_table['TILEID'] == new_tile.tileid) & (exposures_table['EFFTIME_SPEC'] > 0))[0]
    if len(row_index) > 0:
        load_tiles.append(new_tile)
        load_exposures += db.Exposure.convert(exposures_table, row_index=row_index)
        # if (exposures_table[row_index]['MJD'] < 50000).any():
        #     print("WARNING: Invalid MJD values detected for tile {0:d}!".format(new_tile.tileid))
        #     bad_index = np.where((exposures_table['TILEID'] == new_tile.tileid) & (exposures_table['EFFTIME_SPEC'] > 0) & (exposures_table['MJD'] < 50000))[0]
        #     for row in exposures_table[['EXPID', 'NIGHT', 'MJD', 'EFFTIME_SPEC']][bad_index]:
        #         raw_data_file = os.path.join(os.environ['DESI_SPECTRO_DATA'], "{0:08d}".format(row['NIGHT']), "{0:08d}".format(row['EXPID']), "desi-{0:08d}.fits.fz".format(row['EXPID']))
        #         with fits.open(raw_data_file, mode='readonly') as hdulist:
        #             mjd_obs = hdulist['SPEC'].header['MJD-OBS']
        #             print("WARNING: tile {0:d} exposure {1:d} has MJD-OBS = {2:f} in {3}!".format(new_tile.tileid, row['EXPID'], mjd_obs, raw_data_file))
    else:
        print("ERROR: No valid exposures found for tile {0:d}, even though EFFTIME_SPEC == {1:f}!".format(new_tile.tileid, new_tile.efftime_spec))
        bad_index = np.where((exposures_table['TILEID'] == new_tile.tileid))[0]
        print(exposures_table[['EXPID', 'NIGHT', 'MJD', 'EFFTIME_SPEC']][bad_index])
        bad_tiles.append(new_tile)
# load_exposures

CPU times: user 11.3 ms, sys: 0 ns, total: 11.3 ms
Wall time: 9.43 ms


In [10]:
%%time
load_frames = list()
for exposure in load_exposures:
    row_index = np.where(frames_table['EXPID'] == exposure.expid)[0]
    assert len(row_index) > 0
    load_frames += db.Frame.convert(frames_table, row_index=row_index)
# load_frames

CPU times: user 55.5 ms, sys: 0 ns, total: 55.5 ms
Wall time: 54.3 ms


In [11]:
%%time
try:
    db.dbSession.add_all(load_tiles)
    db.dbSession.commit()
except IntegrityError as exc:
    print(exc.args[0])
    db.dbSession.rollback()

CPU times: user 1.98 ms, sys: 2.26 ms, total: 4.24 ms
Wall time: 29.7 ms


In [12]:
%%time
try:
    db.dbSession.add_all(load_exposures)
    db.dbSession.commit()
except IntegrityError as exc:
    print(exc.args[0])
    db.dbSession.rollback()

CPU times: user 4.58 ms, sys: 473 µs, total: 5.06 ms
Wall time: 14.9 ms


In [13]:
%%time
try:
    db.dbSession.add_all(load_frames)
    db.dbSession.commit()
except IntegrityError as exc:
    print(exc.args[0])
    db.dbSession.rollback()

CPU times: user 13.4 ms, sys: 0 ns, total: 13.4 ms
Wall time: 26.6 ms


## Load photometry for the tile

When tractor photometry is written out by John Moustakas' VAC code, only objects with `brickname != ''` are written.

In [14]:
tile_index = 0

In [15]:
load_tiles[tile_index] #, load_tiles[tile_index + 1], load_tiles[-2], load_tiles[-1]

Tile(tileid=80615)

In [16]:
%%time
potential_targets_table = t.potential_targets(load_tiles[tile_index].tileid)
# potential_targets_table

CPU times: user 269 ms, sys: 40.7 ms, total: 310 ms
Wall time: 524 ms


In [17]:
%%time
potential_cat = t.potential_photometry(load_tiles[tile_index], potential_targets_table)
# potential_cat

CPU times: user 51.2 ms, sys: 1.78 ms, total: 53 ms
Wall time: 98.9 ms


In [18]:
%%time
potential_targetphot = t.targetphot(potential_cat)

CPU times: user 2.79 s, sys: 1.79 s, total: 4.57 s
Wall time: 8.21 s


In [19]:
%%time
potential_tractorphot = t.tractorphot(potential_cat)

CPU times: user 6min 9s, sys: 13 s, total: 6min 22s
Wall time: 6min 48s


In [20]:
%%time
load_photometry = t.load_photometry(potential_tractorphot)

INFO:tile.py:178:load_photometry: Loaded 20343 rows of Photometry data.
CPU times: user 10.9 s, sys: 383 ms, total: 11.2 s
Wall time: 17.4 s


### Load photometry, such as it is, for objects that are not in the tractor catalog

In [21]:
%%time
load_targetphot = t.load_targetphot(potential_targetphot, load_photometry)

INFO:tile.py:234:load_targetphot: Loaded 613 rows of Photometry data (from targeting).
CPU times: user 405 ms, sys: 0 ns, total: 405 ms
Wall time: 565 ms


### Load the target table

In [22]:
%%time
load_target = t.load_target(load_tiles[tile_index], potential_targetphot)

INFO:tile.py:260:load_target: Loaded 20956 rows of Target data.
CPU times: user 3.35 s, sys: 24.3 ms, total: 3.38 s
Wall time: 5.37 s


## Load tile/cumulative redshifts

Need a way to compute "best" spectra as new tiles are added. There are a lot of columns that come from other sources here, so need to track these down.

In [ ]:
%%time
load_ztile = t.load_redshift(load_tiles[tile_index])

In [29]:
from redrock.external.desi import DistTargetsDESI
redrock_file, redrock_exists = findfile('redrock_tile', groupname='cumulative', tile=load_tiles[tile_index].tileid, spectrograph=0, night=load_tiles[tile_index].lastnight, readonly=True, return_exists=True)
print(redrock_file, redrock_exists)
print(redrock_file.replace('redrock', 'zbest'), os.path.exists(redrock_file.replace('redrock', 'zbest')))
print(redrock_file.replace('redrock', 'spectra'), os.path.exists(redrock_file.replace('redrock', 'spectra')))
dtd = DistTargetsDESI(redrock_file.replace('redrock', 'spectra'), coadd=True)

/dvs_ro/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/80615/20201216/redrock-0-80615-thru20201216.fits False
/dvs_ro/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/80615/20201216/zbest-0-80615-thru20201216.fits True
/dvs_ro/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/80615/20201216/spectra-0-80615-thru20201216.fits True


KeyError: 'TARGETID'

## Load fiberassign and potential

In [ ]:
%%time
load_fiberassign, load_potential = t.load_fiberassign(load_tiles[tile_index])

## Recompute global values

The global values are the primary classification and number of spectra.

In [ ]:
t.update_primary()

## q3c Update

`tile`, `exposure`, `photometry`, `fiberassign`

In [ ]:
t.update_q3c()